In [1]:
def cross(a,b):
    return [s+t for s in a for t in b]

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return                

In [2]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Input: A grid in string form.
    Output: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

In [3]:
def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [4]:
def naked_twins(values):
    for unit in unitlist:
        pairs_keys = [box for box in unit if len(values[box]) == 2]
        pairs_values = [values[box] for box in unit if len(values[box]) == 2]
        if len(pairs_values)%2 == 0 and len(pairs_values) >1:
            duplicates = [value for value in pairs_values if pairs_values.count(value) > 1]
#             nonpair_keys = [box for box in unit if len(values[box]) != 2]
            twins_values = values
            for dub in duplicates:
                twins = dub
                for digit in twins:
                    for box in unit:
                        if twins_values[box] != dub: 
                            values[box] = twins_values[box].replace(digit,'')
    return values

In [12]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = naked_twins(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [6]:
def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [7]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [8]:
boxes = cross(rows, cols)

In [24]:
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diagnal1 = [[a[0]+a[1] for a in zip(rows,cols)]]
diagnal2 = [[a[0]+a[1] for a in zip(rows,cols[::-1])]]

unitlist = row_units + column_units + square_units + diagnal1 + diagnal2
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [13]:
grid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid)

In [14]:
values = eliminate(values)
values = only_choice(values)

In [42]:
display(fix_values)

 2345689  2345689   34589  | 23456789  235678  2345689 |  24567      4        1    
  123456   123456    345   | 1234567   123567   123456 |    8        9       567   
 1245689  1245689     7    | 1245689   12568   1245689 |   2456      3        56   
---------------------------+---------------------------+---------------------------
 1234569  12345679   3459  |  123569   12356    123569 |  14579      8       3579  
  135689   135689    3589  |  135689     4        7    |   159       15       2    
 1234589  12345789  34589  |  123589   12358    123589 |  14579      6       3579  
---------------------------+---------------------------+---------------------------
    7      34589      2    |  134568   13568    134568 |   1569      15      5689  
   3589     3589      1    |  235678   235678   23568  |  25679      27       4    
   458      458       6    |  124578     9      12458  |    3        27      578   


In [16]:
values = reduce_puzzle(values)
display(values)

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


In [17]:
values = search(values)

In [18]:
display(values)

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 


Test

In [25]:
grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

In [10]:
grid = '9.1....8.8.5.7..4.2.4....6...7......5..............83.3..6......9................'

In [26]:
# grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
test_values = grid_values(grid)
# display(reduce_puzzle(test_values))
display(search(reduce_puzzle(test_values)))

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


In [391]:
before_naked_twins_1 = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}

In [355]:
possible_solutions_1 = [
        {'G7': '6', 'G6': '3', 'G5': '2', 'G4': '9', 'G3': '1', 'G2': '8', 'G1': '7', 'G9': '5', 'G8': '4', 'C9': '1',
         'C8': '5', 'C3': '8', 'C2': '237', 'C1': '23', 'C7': '9', 'C6': '6', 'C5': '37', 'A4': '2357', 'A9': '8',
         'A8': '6', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'F6': '125', 'F7': '35', 'F8': '9',
         'F9': '7', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'C4': '4',
         'B8': '3', 'B9': '4', 'I9': '9', 'I8': '7', 'I1': '23', 'I3': '23', 'I2': '6', 'I5': '5', 'I4': '8', 'I7': '1',
         'I6': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'E8': '1', 'A7': '27', 'A6': '257', 'E5': '347',
         'E4': '6', 'E7': '345', 'E6': '579', 'E1': '8', 'E3': '79', 'E2': '37', 'H8': '2', 'H9': '3', 'H2': '9',
         'H3': '5', 'H1': '4', 'H6': '17', 'H7': '8', 'H4': '17', 'H5': '6', 'D8': '8', 'D9': '6', 'D6': '279',
         'D7': '34', 'D4': '237', 'D5': '347', 'D2': '1', 'D3': '79', 'D1': '5'},
        {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8', 'H5': '6', 'F9': '7',
         'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8', 'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23',
         'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5', 'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9',
         'A4': '2357', 'A7': '27', 'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
         'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2', 'F6': '125',
         'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '79', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235',
         'F5': '8', 'E2': '3', 'F7': '35', 'F8': '9', 'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17',
         'D3': '79', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6',
         'D6': '279', 'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
        ]

In [392]:
before_naked_twins_2 = {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9',
                            'A9': '1', 'B1': '6', 'B2': '9', 'B3': '8', 'B4': '4', 'B5': '37', 'B6': '1', 'B7': '237',
                            'B8': '5', 'B9': '237', 'C1': '23', 'C2': '5', 'C3': '1', 'C4': '23', 'C5': '379',
                            'C6': '2379', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8', 'D2': '17', 'D3': '9',
                            'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
                            'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9',
                            'F1': '4', 'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6',
                            'F8': '8', 'F9': '257', 'G1': '1', 'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345',
                            'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7', 'H2': '2', 'H3': '4', 'H4': '9',
                            'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3', 'I3': '5',
                            'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'}

In [16]:
possible_solutions_2 = [
        {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9', 'A9': '1', 'B1': '6',
         'B2': '9', 'B3': '8', 'B4': '4', 'B5': '37', 'B6': '1', 'B7': '237', 'B8': '5', 'B9': '237', 'C1': '23',
         'C2': '5', 'C3': '1', 'C4': '23', 'C5': '79', 'C6': '79', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8',
         'D2': '17', 'D3': '9', 'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
         'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9', 'F1': '4',
         'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6', 'F8': '8', 'F9': '257', 'G1': '1',
         'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345', 'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7',
         'H2': '2', 'H3': '4', 'H4': '9', 'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3',
         'I3': '5', 'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'},
        {'A1': '23', 'A2': '4', 'A3': '7', 'A4': '6', 'A5': '8', 'A6': '5', 'A7': '23', 'A8': '9', 'A9': '1', 'B1': '6',
         'B2': '9', 'B3': '8', 'B4': '4', 'B5': '3', 'B6': '1', 'B7': '237', 'B8': '5', 'B9': '237', 'C1': '23',
         'C2': '5', 'C3': '1', 'C4': '23', 'C5': '79', 'C6': '79', 'C7': '8', 'C8': '6', 'C9': '4', 'D1': '8',
         'D2': '17', 'D3': '9', 'D4': '1235', 'D5': '6', 'D6': '237', 'D7': '4', 'D8': '27', 'D9': '2357', 'E1': '5',
         'E2': '6', 'E3': '2', 'E4': '8', 'E5': '347', 'E6': '347', 'E7': '37', 'E8': '1', 'E9': '9', 'F1': '4',
         'F2': '17', 'F3': '3', 'F4': '125', 'F5': '579', 'F6': '279', 'F7': '6', 'F8': '8', 'F9': '257', 'G1': '1',
         'G2': '8', 'G3': '6', 'G4': '35', 'G5': '345', 'G6': '34', 'G7': '9', 'G8': '27', 'G9': '27', 'H1': '7',
         'H2': '2', 'H3': '4', 'H4': '9', 'H5': '1', 'H6': '8', 'H7': '5', 'H8': '3', 'H9': '6', 'I1': '9', 'I2': '3',
         'I3': '5', 'I4': '7', 'I5': '2', 'I6': '6', 'I7': '1', 'I8': '4', 'I9': '8'}
    ]

In [10]:
before_naked_twins_3 = {
        "A1": "2468",   "A2": "9",      "A3": "1",      "A4": "5",     "A5": "3468",    "A6": "7",       "A7": "2346",   "A8": "2346",    "A9": "2346",
        "B1": "5",      "B2": "2",      "B3": "3",      "B4": "46",    "B5": "46",      "B6": "1",       "B7": "23468",  "B8": "8",       "B9": "9",
        "C1": "4678",   "C2": "68",     "C3": "4678",   "C4": "9",     "C5": "2",       "C6": "368",     "C7": "13456",  "C8": "13456",   "C9": "13456",
        "D1": "123689", "D2": "4",      "D3": "25689",  "D4": "23678", "D5": "1356789", "D6": "23568",   "D7": "123568", "D8": "1235679", "D9": "1235678",
        "E1": "123689", "E2": "123568", "E3": "25689",  "E4": "234678","E5": "13456789","E6": "23568",   "E7": "1234568","E8": "12345679","E9": "12345678",
        "F1": "123689", "F2": "7",      "F3": "25689",  "F4": "23468", "F5": "1345689", "F6": "23568",   "F7": "1234568","F8": "1234569", "F9": "1234568",
        "G1": "1234678","G2": "123568", "G3": "245678", "G4": "23678", "G5": "35678",   "G6": "9",       "G7": "1234568","G8": "1234567", "G9": "12345678",
        "H1": "2346789","H2": "23568",  "H3": "2456789","H4": "1",     "H5": "35678",   "H6": "23568",   "H7": "234568", "H8": "234567",  "H9": "2345678",
        "I1": "123678", "I2": "123568", "I3": "25678",  "I4": "23678", "I5": "35678",   "I6": "4",       "I7": "9",      "I8": "123567",  "I9": "1235678"
    }

In [61]:
for unit in unitlist:
    pairs_keys = [box for box in unit if len(fix_values[box]) == 2]
    pairs_values = [fix_values[box] for box in unit if len(fix_values[box]) == 2]
    print(unit)
    print(pairs_keys,':',pairs_values)
    if len(pairs_values) >1:
        duplicates = [value for value in pairs_values if pairs_values.count(value) > 1]
        nonpair_keys = [box for box in unit if len(fix_values[box]) != 2]
        twins_values = fix_values
        print(duplicates)
        print(nonpair_keys)
        for dub in duplicates:
            twins = dub
            for digit in twins:
                print(digit)
                for box in unit:
                    if twins_values[box] != dub: 
                        twins_values[box] = twins_values[box].replace(digit,'')
                        print(digit, box, twins_values[box])
    print('\n')
display(twins_values)
# return twins_values

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
[] : []


['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']
[] : []


['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
['C9'] : ['56']


['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9']
[] : []


['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9']
['E8'] : ['15']


['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9']
[] : []


['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9']
['G8'] : ['15']


['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9']
['H8'] : ['27']


['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']
['I8'] : ['27']


['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
[] : []


['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2']
[] : []


['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3']
[] : []


['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4']
[] : []


['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5']
[] : []


['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', '

In [60]:
fix_values = {"G7": "1569", "G6": "134568", "G5": "13568", "G4": "134568", "G3":
"2", "G2": "34589", "G1": "7", "G9": "5689", "G8": "15", "C9": "56",
"C8": "3", "C3": "7", "C2": "1245689", "C1": "1245689", "C7": "2456",
"C6": "1245689", "C5": "12568", "C4": "1245689", "E5": "4", "E4":
"135689", "F1": "1234589", "F2": "12345789", "F3": "34589", "F4":
"123589", "F5": "12358", "F6": "123589", "F7": "14579", "F8": "6",
"F9": "3579", "B4": "1234567", "B5": "123567", "B6": "123456", "B7":
"8", "B1": "123456", "B2": "123456", "B3": "345", "B8": "9", "B9":
"567", "I9": "578", "I8": "27", "I1": "458", "I3": "6", "I2": "458",
"I5": "9", "I4": "124578", "I7": "3", "I6": "12458", "A1": "2345689",
"A3": "34589", "A2": "2345689", "E9": "2", "A4": "23456789", "A7":
"24567", "A6": "2345689", "A9": "1", "A8": "4", "E7": "159", "E6":
"7", "E1": "135689", "E3": "3589", "E2": "135689", "E8": "15", "A5":
"235678", "H8": "27", "H9": "4", "H2": "3589", "H3": "1", "H1":
"3589", "H6": "23568", "H7": "25679", "H4": "235678", "H5": "235678",
"D8": "8", "D9": "3579", "D6": "123569", "D7": "14579", "D4":
"123569", "D5": "12356", "D2": "12345679", "D3": "3459", "D1":
"1234569"}

In [59]:
display(search(twins_values))

TypeError: 'NoneType' object is not subscriptable

In [46]:
display(naked_twins(fix_values))

  235689   235689    3589  | 2356789   235678   235689 |    27       4        1    
  123456   12356     345   | 1234567   123567   123456 |    8        9        7    
  12489    12489      7    |  12489     128     12489  |    56       3        56   
---------------------------+---------------------------+---------------------------
 1234569  12345679   3459  |  123569   12356     356   |  14579      8       3579  
  135689   135689    3589  |  135689     4        7    |   159       15       2    
 1234589  12345789  34589  |   358     12358    123589 |  14579      6       3579  
---------------------------+---------------------------+---------------------------
    7      34589      2    |  134568   13568    134568 |   1569      15      5689  
   3589     358       1    |  235678   235678   23568  |  25679      2        4    
    58      458       6    |  124578     9      12458  |    3        27       58   


In [44]:
display(eliminate(fix_values))

  235689   235689    3589  | 2356789   235678   235689 |   2567      4        1    
  123456   12356     345   | 1234567   123567   123456 |    8        9       567   
 1245689  1245689     7    | 1245689   12568   1245689 |    56       3        56   
---------------------------+---------------------------+---------------------------
 1234569  12345679   3459  |  123569   12356     356   |  14579      8       3579  
  135689   135689    3589  |  135689     4        7    |   159       15       2    
 1234589  12345789  34589  |   358     12358    123589 |  14579      6       3579  
---------------------------+---------------------------+---------------------------
    7      34589      2    |  134568   13568    134568 |   1569      15      5689  
   3589     358       1    |  235678   235678   23568  |  25679      2        4    
    58      458       6    |  124578     9      12458  |    3        27       58   


In [13]:
solution_twins_3 = [{
        "A1": "2468",    "A2": "9",      "A3": "1",      "A4": "5",     "A5": "3468",    "A6": "7",     "A7": "2346",   "A8": "2346",     "A9": "2346",
        "B1": "5",       "B2": "2",      "B3": "3",      "B4": "46",    "B5": "46",      "B6": "1",     "B7": "238",    "B8": "8",         "B9": "9",
        "C1": "4678",    "C2": "68",     "C3": "4678",   "C4": "9",     "C5": "2",       "C6": "368",   "C7": "13456",  "C8": "13456",     "C9": "13456",
        "D1": "123689",  "D2": "4",      "D3": "25689",  "D4": "23678", "D5": "1356789", "D6": "23568", "D7": "123568",  "D8": "1235679",  "D9": "1235678",
        "E1": "123689",  "E2": "123568", "E3": "25689",  "E4": "234678","E5": "13456789", "E6": "23568","E7": "1234568","E8": "12345679", "E9": "12345678",
        "F1": "123689",  "F2": "7",      "F3": "25689",  "F4": "23468", "F5": "1345689",  "F6": "23568","F7": "1234568","F8": "1234569",  "F9": "1234568",
        "G1": "1234678", "G2": "123568", "G3": "245678", "G4": "23678", "G5": "35678",    "G6": "9",    "G7": "1234568","G8": "1234567",  "G9": "12345678",
        "H1": "2346789", "H2": "23568",  "H3": "2456789","H4": "1",     "H5": "35678",    "H6": "23568","H7": "234568", "H8": "234567",   "H9": "2345678",
        "I1": "123678",  "I2": "123568", "I3": "25678",  "I4": "23678", "I5": "35678",    "I6": "4",    "I7": "9",      "I8": "123567",   "I9": "1235678"
    }]

In [401]:
%timeit naked_twins(before_naked_twins_1)

The slowest run took 4.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 284 µs per loop


In [334]:
possible_solutions_2[1] == check_sudoku

True

Timeit!

In [206]:
grid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
%timeit only_choice(eliminate(grid_values(grid)))

The slowest run took 4.36 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 575 µs per loop


In [207]:
grid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
%timeit only_c(eliminate(grid_values(grid)))

1000 loops, best of 3: 543 µs per loop
